# Election Statis

$H_0$: There is no relationship between campaign funding and winning

$H_1$: There is a relationship between campaign funding and winning

I suspect there is no relationship anymore, and that it would be a negative relationship if you look at only recent data.

In [ ]:
import numpy as np
import pandas as pd